In [40]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

import couponProjectUtils as proj_utils

from sklearn.preprocessing import StandardScaler

from imblearn.over_sampling import SMOTE

## models
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

## to make it possible to display multiple output inside one cell 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.options.display.max_rows = 70
pd.set_option('display.float_format', lambda x: '%9.8f' % x)

# Use best encoding for coupon type: Restaurant(<20)

In [3]:
## load data, split Xy, upsample with SMOTE, split train/test
data = pd.read_csv('data/RestaurantLessThan20_data_encoded-time_nominal.csv')

## define X and y 
X = data.drop('Y', axis=1).reset_index(drop=True)
y = data.Y

## Handle imbalance
smote = SMOTE(random_state=42)

orig_vc = y.value_counts()
print(f"Orig: {100* orig_vc[1]/(orig_vc.sum()):.2f}% Yes")

X_sm, y_sm = smote.fit_resample(X, y)

## Data splitting train/test
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.20, random_state=42)

print("After SMOTE upsampling")
train_vc = y_train.value_counts()
print(f"\tNum training samples: {X_train.shape[0]:,} ({100* train_vc[1]/(train_vc.sum()):.2f}% yes)")
test_vc = y_test.value_counts()
print(f"\tNum testing samples:  {X_test.shape[0]:,} ({100* test_vc[1]/(test_vc.sum()):.2f}% yes)")

Orig: 70.78% Yes
After SMOTE upsampling
	Num training samples: 3,147 (49.98% yes)
	Num testing samples:  787 (50.06% yes)


In [4]:
## Replace the unknown values (previously encoded as 0.0) using the most frequent value seen in the TRAINING data
X_train, X_test, train_modes = proj_utils.replace_unknown_with_mode(X_train, X_test, 
                                                                    outfile_prefix='phase2-RestaurantLessThan20')

## Testing RandomForestClassifier with Grid Search Cross-validation


In [5]:
cols_to_view = ['rank_test_score', 'param_max_features', 'param_min_samples_split', 'param_n_estimators', 
                'mean_test_score', 'std_test_score', 'mean_train_score', 'std_train_score']

param_grid={'max_features': [2, 3, 4, 5, 6, 7, 8, 9, 10],
            'min_samples_split': [2, 3, 4, 5],
            'n_estimators': [100, 150, 200]}

clf = RandomForestClassifier(oob_score=True, random_state=42)

grid_search1 = GridSearchCV(clf, param_grid, scoring='roc_auc', cv=5, return_train_score=True, n_jobs=4)
grid_search1.fit(X_train.values, y_train.values)

cv_results1 = pd.DataFrame(grid_search1.cv_results_)
best_model = grid_search1.best_estimator_
best_model

GridSearchCV(cv=5,
             estimator=RandomForestClassifier(oob_score=True, random_state=42),
             n_jobs=4,
             param_grid={'max_features': [2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'min_samples_split': [2, 3, 4, 5],
                         'n_estimators': [100, 150, 200]},
             return_train_score=True, scoring='roc_auc')

RandomForestClassifier(max_features=2, min_samples_split=3, n_estimators=200,
                       oob_score=True, random_state=42)

In [6]:
print(f"Best model oob score: {best_model.oob_score_}")
y_test_score = best_model.predict_proba(X_test)[:, 1]
print(f"ROC AUC: TEST {roc_auc_score(y_test, y_test_score):.6f}\n")

Best model oob score: 0.8554178582777248
ROC AUC: TEST 0.948076



In [7]:
cv_results1.loc[:, cols_to_view].sort_values('rank_test_score').head(10)

,rank_test_score,param_max_features,param_min_samples_split,param_n_estimators,mean_test_score,std_test_score,mean_train_score,std_train_score
5,1,2,3,200,0.92314286,0.01022014,0.99996642,0.00002670
13,2,3,2,150,0.92308962,0.00974555,0.99999950,0.00000038
2,3,2,2,200,0.92304418,0.01050525,0.99999950,0.00000038
26,4,4,2,200,0.92258975,0.01063408,0.99999924,0.00000055
14,5,3,2,200,0.92241815,0.01005520,0.99999950,0.00000038
1,6,2,2,150,0.92206499,0.01028609,0.99999950,0.00000038
25,7,4,2,150,0.92186618,0.01059413,0.99999937,0.00000040
29,8,4,3,200,0.92178222,0.01052859,0.99997236,0.00001771
4,9,2,3,150,0.92165133,0.01085133,0.99996857,0.00002374
17,10,3,3,200,0.92120926,0.01032968,0.99996933,0.00002265


In [8]:
featimp_df = pd.DataFrame({'feature_name': X.columns, 'importance': best_model.feature_importances_})
featimp_df.sort_values('importance', ascending=False).head(10)

,feature_name,importance
9,income,0.06064292
7,age,0.05777058
11,minsToCouponDest,0.05605809
13,CoffeeHouse,0.05391411
8,education,0.05053328
15,RestaurantLessThan20,0.05028352
14,CarryAway,0.04885656
12,Bar,0.04666874
16,Restaurant20To50,0.04591961
17,destination_No Urgent Place,0.04321239


In [9]:
import pickle

with open('model/phase2/RestaurantLessThan20_GridSearchCV_RFC.pickle', "wb") as f:
    pickle.dump(grid_search1,f)
    
with open('model/phase2/RestaurantLessThan20_best_model_RFC.pickle', "wb") as f:
    pickle.dump(best_model,f)

In [45]:
## save train/test info to file
Xy_tt = {'X_train': X_train, 'X_test': X_test, 'y_train': y_train, 'y_test': y_test}

for k in Xy_tt.keys():
    fname = f'data/RestaurantLessThan20_{k}_phase2.csv'
    print(f'Saving file: {fname}')
    Xy_tt[k].to_csv(fname, index=False)

Saving file: data/RestaurantLessThan20_X_train_phase2.csv
Saving file: data/RestaurantLessThan20_X_test_phase2.csv
Saving file: data/RestaurantLessThan20_y_train_phase2.csv
Saving file: data/RestaurantLessThan20_y_test_phase2.csv


# KNN

In [46]:
param_grid={'n_neighbors': [x for x in range(1,12)]}

clf = KNeighborsClassifier()

grid_search_knn = GridSearchCV(clf, param_grid, scoring='roc_auc', cv=5, return_train_score=True, n_jobs=4)
grid_search_knn.fit(X_train.values, y_train.values)

knn_cols_to_view = ['rank_test_score', 'param_n_neighbors', 
                    'mean_test_score', 'std_test_score', 'mean_train_score', 'std_train_score']

cv_results_knn = pd.DataFrame(grid_search_knn.cv_results_)
cv_results_knn.loc[:, knn_cols_to_view].sort_values('rank_test_score').head(5)

best_model_knn = grid_search_knn.best_estimator_
best_model
y_test_score_knn = best_model_knn.predict_proba(X_test)[:, 1]
print(f"ROC AUC: TEST {roc_auc_score(y_test, y_test_score_knn):.6f}\n")

GridSearchCV(cv=5, estimator=KNeighborsClassifier(), n_jobs=4,
             param_grid={'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]},
             return_train_score=True, scoring='roc_auc')

,rank_test_score,param_n_neighbors,mean_test_score,std_test_score,mean_train_score,std_train_score
2,1,3,0.87082586,0.01132091,0.97385532,0.00097652
3,2,4,0.87055904,0.01023428,0.96045408,0.00159159
4,3,5,0.86538159,0.00881917,0.94885161,0.00279101
1,4,2,0.86448741,0.01457504,0.98625934,0.00057474
5,5,6,0.86008357,0.00930263,0.93685286,0.00179686


RandomForestClassifier(max_features=2, min_samples_split=3, n_estimators=200,
                       oob_score=True, random_state=42)

ROC AUC: TEST 0.910777



In [47]:
with open('model/phase2/RestaurantLessThan20_GridSearchCV_KNN.pickle', "wb") as f:
    pickle.dump(grid_search_knn,f)
    
with open('model/phase2/RestaurantLessThan20_best_model_KNN.pickle', "wb") as f:
    pickle.dump(best_model_knn,f)

# Logistic regression


In [48]:
## standardization
std_scaler=StandardScaler().fit(X_train) ## first fit the model on the training data

## now std scaler transform
X_train_scaled = std_scaler.transform(X_train)
X_test_scaled = std_scaler.transform(X_test)

## apply regression
liblinear_model = LogisticRegression(solver='liblinear', random_state=42)
liblinear_model.fit(X_train_scaled, y_train)   # train model by fitting to training data

## use cv to assess generalization
cross_val_scores = cross_val_score(liblinear_model, X_train, y_train, scoring='roc_auc', cv=5)
print(f"5-fold CV ROC AUC: mean: {np.mean(cross_val_scores):.6f} std dev:{np.std(cross_val_scores):.6f}")
    
# use trained model to get predicted target var
y_test_score_ll = liblinear_model.predict_proba(X_test_scaled)[:, 1]

print(f"ROC AUC: TEST {roc_auc_score(y_test, y_test_score_ll)}\n")

## apply regression again, using a different solver
lbfgs_model = LogisticRegression(solver='lbfgs', random_state=42)
lbfgs_model.fit(X_train_scaled, y_train)   # train model by fitting to training data

## use cv to assess generalization
cross_val_scores = cross_val_score(lbfgs_model, X_train, y_train, scoring='roc_auc', cv=5)
print(f"5-fold CV ROC AUC: mean: {np.mean(cross_val_scores):.6f} std dev:{np.std(cross_val_scores):.6f}")

# use trained model to get predicted target var
y_test_score_lb = lbfgs_model.predict_proba(X_test_scaled)[:, 1]

print(f"ROC AUC: TEST {roc_auc_score(y_test, y_test_score_lb)}\n")

LogisticRegression(random_state=42, solver='liblinear')

5-fold CV ROC AUC: mean: 0.858770 std dev:0.011154
ROC AUC: TEST 0.8846114103408637



LogisticRegression(random_state=42)

5-fold CV ROC AUC: mean: 0.858712 std dev:0.011014
ROC AUC: TEST 0.8845920357525736



In [50]:
## both LR models have essentially the same score, just save one

with open('model/phase2/RestaurantLessThan20_best_model_LogisticRegression.pickle', "wb") as f:
    pickle.dump(liblinear_model,f)